In [210]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import statsmodels.formula.api as smf

In [211]:
from io import StringIO

## Sales Growth Portfolio Assignment


#### Step 1

Collect data on all stocks that exist in year t and also existed in the previous 3 years. Delete financial firms, since sales may not be a meaningful measure for them.  You should test a trading strategy that starts in 1967, which means that your initial portfolio will consist of firms with sales data in 1964. 

In [279]:
## Bring in Data
sales = pd.read_csv('annualsales.csv')
returns = pd.read_csv('monthlyreturns.csv',parse_dates=True)

In [280]:
#Clean data

In [281]:
returns = returns.query('date > 19661201')
returns = returns.query('SHRCD == 10 or SHRCD == 11')
returns = returns.query("SHRCLS == 'A' or SHRCLS == 'NaN'")

In [282]:
returns = returns.drop(returns[returns.RET == 'C'].index)
returns = returns.drop(returns[returns.RET == 'B'].index)

In [283]:
#Sales
sales = sales.query("fic == 'USA'")
sales['sic_use']= sales['sich']
sales['sic_use'] = sales['sic_use'].fillna(sales['sic'])
sales = sales.query('sic_use < 6000 or sic_use > 6999')
sales = sales.query('exchg != [0, 1, 2, 3, 4, 7, 20]')

In [284]:
#Remove duplicates

In [285]:
sales = sales.drop_duplicates(subset=['GVKEY','fyear'])

In [286]:
#Non-missing non-neg sales

In [287]:
sales = sales.query('sale > 0')
sales = sales.dropna(subset=['sale'])

In [288]:
returns['RET1'] = returns.RET.astype(float)

In [289]:
#returns.describe()

#### Step 2

Sort stocks into 10 groups by the percentage growth in sales over the previous 3 years.

In [315]:
#generate sales growth over past three years

sales1 = sales
sales1['lag'] = sales1.groupby('LPERMNO')['sale'].shift(2)

sales1['growth']=(sales1['sale']/sales1['lag']) - 1 

#only if its been around for at least 3 years
sales1 = sales1.dropna(subset=['growth'])

In [291]:
sales1['growth'].describe()

count    159722.000000
mean          2.132319
std         110.549248
min          -0.999979
25%          -0.007773
50%           0.184237
75%           0.456377
max       21251.000000
Name: growth, dtype: float64

In [292]:
#sales1['port'] = pd.qcut(sales1['growth'],  q = 10, labels = False)
#df1['QuantileRank']= pd.qcut(df1['EnglishScore'],  q = 4, labels = False)

In [293]:
#sales1

In [294]:
dups = sales1.pivot_table(index=['growth'], aggfunc='size')
dups.describe(percentiles=[.25, .5, .999])

count    159340.000000
mean          1.002397
std           0.134140
min           1.000000
25%           1.000000
50%           1.000000
99.9%         2.000000
max          48.000000
dtype: float64

In [295]:
#sales1.loc[sales1.growth > 10000000, 'growth'] = 10000000
#df.loc[df.my_channel > 20000, 'my_channel'] = 0

In [296]:
#sales1['port'] = sales1.groupby('fyear')['growth'].transform(pd.qcut,5, labels=False)
sales1['port'] = sales1.groupby('fyear')['growth'].transform(pd.qcut,10,labels=False, duplicates = 'drop')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [316]:
sales1['growth'].isnull().count()

159722

In [260]:
#dups1 = sales1.pivot_table(index=['port'], aggfunc='size')
#dups1

In [298]:
days = pd.to_datetime(returns['date'], format='%Y%m%d')
returns['year']=pd.DatetimeIndex(days).year
returns['month']=pd.DatetimeIndex(days).month
returns['year_month'] = returns['year']*100 + returns['month']

In [299]:
sales1['year'] = sales['fyear'] - 1
sales1['PERMNO'] = sales1['LPERMNO']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [300]:
#merge
both = pd.merge(returns, sales1, how = "left", on = ['PERMNO', 'year'])

In [301]:
#only if its been around for at least 3 years
both = both.query("growth == growth").reset_index(drop=True)

In [302]:
#Get Market Cap
both['mkt'] = np.abs(both['PRC'])*both['SHROUT']

In [303]:
vw = both.pivot_table('RET1','year_month',columns='port',aggfunc='mean')
#vw = both.pivot_table('RET1','year',columns='port',aggfunc=np.average(vw['RET1'],weights = both['mkt']))
#ret*mktcap/sumofall mktcap's (group)
vw.columns = ['p' + str(int(x)) for x in vw.columns]

In [304]:
def summary(p):
    s = p.describe().T
    s['tstat'] = s['mean']/(s['std']/np.sqrt(s['count']))
    return s[['mean','std','tstat']].T

summary(vw)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
mean,-0.003643,0.001192,0.004300,0.008047,0.012449,0.012700,0.019287,0.019339,0.027003,0.028036
std,0.108373,0.087023,0.081365,0.074049,0.072048,0.067738,0.072593,0.071847,0.079465,0.102668
tstat,-0.840476,0.342321,1.321130,2.716857,4.319764,4.687320,6.642237,6.729270,8.495381,6.804960


In [305]:
vw['LML'] = vw['p9'] - vw['p0']
vw['HalfnHalf'] = vw['p9'] + vw['p8'] + vw['p7'] + vw['p6'] + vw['p5'] - vw['p4'] - vw['p3'] - vw['p2'] - vw['p1'] - vw['p0']
vw['30/30'] = vw['p9'] + vw['p8'] + vw['p7'] - vw['p2'] - vw['p1'] - vw['p0']

In [306]:
summary(vw)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,LML,HalfnHalf,30/30
mean,-0.003643,0.001192,0.004300,0.008047,0.012449,0.012700,0.019287,0.019339,0.027003,0.028036,0.032879,0.086557,0.073719
std,0.108373,0.087023,0.081365,0.074049,0.072048,0.067738,0.072593,0.071847,0.079465,0.102668,0.107781,0.189243,0.160119
tstat,-0.840476,0.342321,1.321130,2.716857,4.319764,4.687320,6.642237,6.729270,8.495381,6.804960,7.601902,11.397971,11.473162


In [307]:
#vw.head(10)
vw['LML'].isnull().sum()

4

In [320]:
vw_early = vw.query('year_month<199900')
vw_late = vw.query('year_month>201000')

In [321]:
summary(vw_early)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,LML,HalfnHalf,30/30
mean,-0.002904,-0.000736,0.005062,0.008752,0.012916,0.012334,0.023146,0.020400,0.030884,0.027332,0.032183,0.095220,0.079188
std,0.114465,0.085965,0.085764,0.076717,0.077373,0.071935,0.078687,0.078743,0.084769,0.109669,0.117567,0.201775,0.170952
tstat,-0.497717,-0.168012,1.158090,2.238364,3.275337,3.364403,5.771665,5.083402,7.148697,4.864645,5.343221,9.211354,9.041640


In [322]:
summary(vw_late)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,LML,HalfnHalf,30/30
mean,-0.008434,0.001979,0.006530,0.010424,0.012452,0.010460,0.013536,0.014110,0.021358,0.020004,0.028438,0.056518,0.055398
std,0.084231,0.062288,0.062562,0.052555,0.051248,0.054782,0.048588,0.050643,0.054623,0.073955,0.080004,0.146240,0.125311
tstat,-1.040596,0.330215,1.084657,2.061220,2.525069,1.984337,2.895159,2.895522,4.063427,2.811047,3.694059,4.016363,4.594239


In [313]:
vw.head(55)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,LML,HalfnHalf,30/30
year_month,,,,,,,,,,,,,
196612,-0.071806,0.008226,-0.007768,-0.074420,0.029309,-0.015981,-0.001810,0.080347,0.158627,0.068076,0.139882,0.405718,0.378398
196701,0.155673,0.199697,0.157179,0.291284,0.126995,0.151830,0.185557,0.274749,0.140565,0.276401,0.120728,0.098274,0.179165
196702,0.020453,-0.006369,0.045669,-0.006845,0.073350,0.129153,-0.034956,0.010510,0.055331,-0.016672,-0.037125,0.017110,-0.010583
196703,0.047318,-0.064103,-0.012795,0.043482,0.148272,0.066947,-0.011905,0.093479,0.066849,0.195860,0.148542,0.249056,0.385768
196704,0.038673,0.187397,0.023120,0.014688,0.068282,-0.020330,0.075059,0.057935,0.108163,0.068984,0.030311,-0.042350,-0.014109
196705,-0.020925,-0.116279,-0.016632,-0.010052,-0.037815,-0.024503,-0.017381,-0.016373,0.033374,0.131433,0.152358,0.308253,0.302270
196706,0.053353,-0.085526,0.045081,0.003511,0.173368,0.088726,0.181660,0.196982,0.166895,-0.023505,-0.076857,0.420971,0.327464
196707,0.064774,0.060144,-0.044256,0.064313,0.098134,0.131147,0.167628,0.149299,0.049090,0.134652,0.069877,0.388706,0.252378
196708,0.100163,0.041096,0.053614,-0.025328,-0.110317,-0.019748,-0.084929,0.022439,-0.058114,0.083561,-0.016602,-0.116019,-0.146987
